In [12]:
import pandas as pd
from reportlab.lib.pagesizes import A4
from reportlab.pdfgen import canvas
from reportlab.lib import colors
from reportlab.lib.units import inch
from reportlab.platypus import Table, TableStyle
from PyPDF2 import PdfMerger
import datetime
import os

# Load Order Data
def load_orders(file_path):
    return pd.read_csv(file_path)

# Generate Invoice with Table
def generate_invoice(order, output_dir="invoices"):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Invoice details
    order_id = order['Order ID']
    customer_name = order['Customer Name']
    product_name = order['Product Name']
    quantity = order['Quantity']
    unit_price = order['Unit Price']
    total_amount = quantity * unit_price
    date_of_purchase = datetime.date.today().strftime("%Y-%m-%d")

    # PDF file path
    pdf_file = os.path.join(output_dir, f"Invoice_{order_id}.pdf")

    # Create PDF
    c = canvas.Canvas(pdf_file, pagesize=A4)
    c.setFont("Helvetica", 12)

    # Header
    c.drawString(100, 750, f"Invoice Number: {order_id}")
    c.drawString(100, 730, f"Date of Purchase: {date_of_purchase}")
    c.drawString(100, 710, f"Customer Name: {customer_name}")

    # Table Data
    data = [
        ["Product Name", "Quantity", "Unit Price", "Total Amount"],
        [product_name, quantity, f"${unit_price:.2f}", f"${total_amount:.2f}"]
    ]

    # Create Table
    table = Table(data, colWidths=[3 * inch, 1 * inch, 1.5 * inch, 1.5 * inch])

    # Table Style
    table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('FONTSIZE', (0, 0), (-1, 0), 12),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
        ('BACKGROUND', (0, 1), (-1, -1), colors.beige),
        ('GRID', (0, 0), (-1, -1), 1, colors.black),
    ]))

    # Position the table on the PDF
    table.wrapOn(c, 400, 200)
    table.drawOn(c, 100, 600)

    c.save()

    return pdf_file

# Merge all invoices into a single PDF
def merge_invoices(invoice_files, output_file="Merged_Invoices.pdf"):
    merger = PdfMerger()
    for pdf in invoice_files:
        merger.append(pdf)
    merger.write(output_file)
    merger.close()

def main():
    # Load order data
    orders = load_orders("orders.csv")

    # Generate individual PDF invoices
    invoice_files = []
    for _, order in orders.iterrows():
        pdf_file = generate_invoice(order)
        invoice_files.append(pdf_file)

    # Merge all invoices into a single PDF
    merge_invoices(invoice_files)
    print("All invoices have been merged into Merged_Invoices.pdf")

if __name__ == "__main__":
    main()


All invoices have been merged into Merged_Invoices.pdf
